<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/Code/ReHealthCo_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo EDA-Prozess

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

Installtion der Package ydata_profiling

In [ ]:
!pip install ydata_profiling

## Importieren der notwendigen Libraries: sqlite3, csv, pandas

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
from pandas.util import hash_pandas_object
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt

### Erstellung einer virtuellen Kopie des GitHub Repositorium DMA2023TeamD in Colab

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD.git

In [ ]:
# Wechsel in den Unterordner csv_data des Github-Projekt-Ordners
%cd /content/DMA2023TeamD/csv_data

In [ ]:
# Auflistung aller Ordnerinhalte
!ls

### Benennung des rhc-DataFrames und Einlesen des entsprechenden anonymisierten csv-Files von Github

In [ ]:
# Pfad-Abkürzung zum Speicherort der anonymisierten csv-Tabelle
csv_path_anonymrhc = "/content/DMA2023TeamD/csv_data"

In [ ]:
# Einlesen der anonymisierten Tabelle als Pandas DataFrame
rhc_df = pd.read_csv(f"{csv_path_anonymrhc}/anonym_rhcpat.csv")

In [ ]:
# Anzeige des DF-Inhalts
rhc_df

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_df.columns.tolist())

In [ ]:
# Diagnosen/Patient
rhc_df[["RHC_PSEUDONYM", "DESC_CON"]].groupby(["RHC_PSEUDONYM"]).size()

In [ ]:
# Anzeige Diagnosen/Patient, absteigend sortiert
diag_anz_df = rhc_df[["RHC_PSEUDONYM", "DESC_CON"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("DESC_CON", ascending=False) 
diag_anz_df.rename(columns={"DESC_CON": "DESC_CON_ANZ"}, inplace=True)
diag_anz_df

In [ ]:
# Spalte Diagnosen/Patient an das gesamte DataFrame hängen
rhc_diagn_df = pd.DataFrame.merge(rhc_df, diag_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left")
rhc_diagn_df

In [ ]:
# Spalte DESC_CON_ANZ verdoppeln
rhc_diagn_df.loc[:, 'DESC_CON_SORT'] = rhc_diagn_df.loc[:, 'DESC_CON_ANZ']
# Anzeige der Kopfspalten des DF
print(rhc_diagn_df.columns.tolist())

In [ ]:
# Ersetzen aller Werte bei Patienten > 2 Diagnosen mit einer 3
rhc_diagn_df['DESC_CON_SORT'].values[rhc_diagn_df['DESC_CON_SORT'] > 2] = 3
rhc_diagn_df

In [ ]:
# neue Spalte HEALTHCARE_EXPENSES uncovered
rhc_diagn_df["HEALTHCARE_EXPENSES_UNCOVERED"] = rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]-rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]
rhc_diagn_df

In [ ]:
# neue Spalte Verhältnis Ausgaben abgedeckt/nicht abgedeckt mit Spaltennahmen VERH_HEALTHCARE_EXPENSES_COV_UNCOV
rhc_diagn_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"] = rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]/rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]
rhc_diagn_df

In [ ]:
# Anzeige DF Patienten/DiagnoseErweiterung sowie Encountbegegnungen, , absteigend sortiert nach Patienten/Diagnose
diagn_enc_df = rhc_df[["RHC_PSEUDONYM", "DESC_CON", "ENCOUNTERCLASS_ENC"]].groupby(["DESC_CON"]).count().sort_values("RHC_PSEUDONYM", ascending=False) 
diagn_enc_df

In [ ]:
# Anzeige DF Patienten/DiagnoseErweiterung sowie Encountbegegnungen, absteigend sortiert nach Patienten/Diagnose
pat_anz_nowellrechnung_df = rhc_df.drop(rhc_df[ (rhc_df['ENCOUNTERCLASS_ENC'] == "wellness") ].index) # Löschen der Encounterbegegnung wellness 
diagn_encnowell_df = pat_anz_nowellrechnung_df[["RHC_PSEUDONYM", "DESC_CON", "ENCOUNTERCLASS_ENC"]].groupby(["DESC_CON"]).count().sort_values("RHC_PSEUDONYM", ascending=False) # Benennung des DF
diagn_encnowell_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
diagn_encnowell_df # Anzeige DF

In [ ]:
# Anzeige Encounterbegegnungen/Patient, absteigend sortiert
rhc_enc_anz_df = rhc_diagn_df[["RHC_PSEUDONYM", "ENCOUNTERCLASS_ENC"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("ENCOUNTERCLASS_ENC", ascending=False) 
rhc_enc_anz_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_ENC_ANZ"}, inplace=True)
rhc_enc_anz_df

In [ ]:
# Spalte Encounterbegegnungen/Patient an das gesamte DataFrame hängen
rhc_all_df = pd.DataFrame.merge(rhc_diagn_df, rhc_enc_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left")
rhc_all_df

In [ ]:
# Ersetzen aller Werte bei Patienten > 2 Encounterbegegnungen mit einer 3 nach Verdopplung
rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ_SORT'] = rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ'] # Spalte ENCOUNTERCLASS_ENC_ANZ verdoppeln
rhc_all_df['ENCOUNTERCLASS_ENC_ANZ'].values[rhc_all_df['ENCOUNTERCLASS_ENC_ANZ'] > 2] = 3
rhc_all_df # Anzeige des Ergebnis

In [ ]:
# Löschen der Encounterbegegnung wellness 
rhc_all_nowell_df = rhc_all_df.drop(rhc_all_df[ (rhc_all_df['ENCOUNTERCLASS_ENC'] == "wellness")
                                                     ].index)
rhc_all_nowell_df.groupby(["ENCOUNTERCLASS_ENC"]).size() # Anzeige der Übersicht

In [ ]:
rhc_all_nowell_df # Anzeige des Ergebnis

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_all_nowell_df.columns.tolist())

In [ ]:
# Gruppierung nach Patienten, diese sonst doppelt gezählt werden
groupbypat_df = rhc_all_nowell_df[['RHC_PSEUDONYM', 'GENDER_PAT', 'HEALTHCARE_EXPENSES_PAT', 'HEALTHCARE_COVERAGE_PAT', 'DATENBANKART', 'LEBENDIG', 'AGE', 'CODE_CON', 'DESC_CON', 'CONDITIONDAUER', 'TOTALCOST_MED', 'ENCOUNTERCLASS_ENC', 'BEHANDLUNGSDAUER', 'DESC_CON_ANZ', 'DESC_CON_SORT', 'ENCOUNTERCLASS_ENC_ANZ']].groupby(["RHC_PSEUDONYM"]).mean().reset_index()
groupbypat_df

In [ ]:
# Patienten kommen doppelt vor!
ProfileReport(rhc_all_nowell_df)

In [ ]:
# Profilreporting nach Groupierung auf den Patienten
ProfileReport(groupbypat_df)

In [ ]:
anz_pat_dign_enc_df = diagn_encnowell_df.reset_index()
anz_pat_dign_enc_df.rename(columns={"RHC_PSEUDONYM": "PAT_ANZ", "DESC_CON": "DIAGNOSEN", "ENCOUNTERCLASS_NOWELL": "ENC_BEGEG_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
ProfileReport(anz_pat_dign_enc_df)

# Boxplot and Remove Outlier 
(Extremwerte rausnehmen)

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_all_nowell_df.columns.tolist())

In [ ]:
sns.boxplot(x=groupbypat_df["HEALTHCARE_EXPENSES_PAT"])

In [ ]:
rhc_rein01_df = rhc_all_nowell_df[rhc_all_nowell_df["HEALTHCARE_EXPENSES_PAT"]<1500000]
sns.boxplot(x=rhc_rein01_df["HEALTHCARE_EXPENSES_PAT"])

In [ ]:
sns.boxplot(x=rhc_rein01_df["AGE"])

In [ ]:
rhc_rein02_df = rhc_rein01_df[rhc_rein01_df["AGE"]<100]
sns.boxplot(x=rhc_rein01_df["AGE"])

In [ ]:
sns.boxplot(x=rhc_rein02_df["HEALTHCARE_COVERAGE_PAT"])

In [ ]:
rhc_rein03_df = rhc_rein02_df[rhc_rein02_df["HEALTHCARE_COVERAGE_PAT"]<20000]
sns.boxplot(x=rhc_rein03_df["HEALTHCARE_COVERAGE_PAT"])

In [ ]:
sns.boxplot(x=rhc_rein03_df["CONDITIONDAUER"])

In [ ]:
rhc_rein04_df = rhc_rein03_df[rhc_rein03_df["CONDITIONDAUER"]<5000]
sns.boxplot(x=rhc_rein03_df["CONDITIONDAUER"])

In [ ]:
sns.boxplot(x=rhc_rein04_df["TOTALCOST_MED"])

In [ ]:
rhc_rein05_df = rhc_rein04_df[rhc_rein04_df["TOTALCOST_MED"]<175000]
sns.boxplot(x=rhc_rein04_df["TOTALCOST_MED"])

In [ ]:
sns.boxplot(x=rhc_rein05_df["BEHANDLUNGSDAUER"])

In [ ]:
rhc_rein06_df = rhc_rein05_df[rhc_rein05_df["BEHANDLUNGSDAUER"]<350]
sns.boxplot(x=rhc_rein05_df["CONDITIONDAUER"])

In [ ]:
sns.boxplot(x=rhc_rein06_df["DESC_CON_ANZ"])

In [ ]:
rhc_rein_df = rhc_rein06_df[rhc_rein06_df["DESC_CON_ANZ"]<50]
sns.boxplot(x=rhc_rein_df["DESC_CON_ANZ"])

### von Extremwerten bereinigtes DF

In [ ]:
rhc_rein_df

In [ ]:
# distribution of GENDER attribute ohne Extremwerten
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["GENDER_PAT"]]))

In [ ]:
# distribution of GENDER attribute mit Extremwerten
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["GENDER_PAT"]]))

In [ ]:
# distribution of DESCRIPTION-Anzahl
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["DESC_CON_ANZ"]]))

In [ ]:
# Verteilung Behandlungsdauer in Stunden
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["BEHANDLUNGSDAUER"]]))

In [ ]:
# Verteilung Conditiondauer in Stunden
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["CONDITIONDAUER"]]))

In [ ]:
# Verteilung der Erkrankungen im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["DESC_CON"]]))

In [ ]:
# Verteilung der Erkrankungen im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_all_nowell_df[["DESC_CON"]]))

In [ ]:
# Verteilung Diagnosen/Patient
sns.countplot(x="variable", hue= "value", data=pd.melt(diagn_enc_df[["RHC_PSEUDONYM"]]))

In [ ]:
# Verteilung Diagnosen/Encounterbegenung
sns.countplot(x="variable", hue= "value", data=pd.melt(diagn_enc_df[["ENCOUNTERCLASS_ENC"]]))

In [ ]:
# Histogramm HEALTHCARE_EXPENSES_UNCOVERED im nicht bereinigten DF
rhc_all_nowell_df["HEALTHCARE_EXPENSES_UNCOVERED"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm Verhätnis Kosten gedeckt/nicht abgedeckt im nicht bereinigten DF
rhc_diagn_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')
# rhc_diagn_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"] = rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]/rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]

In [ ]:
# Histogramm Alter bereinigtes DF
rhc_rein_df["AGE"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm Alter nicht bereinigtes DF
rhc_all_nowell_df["AGE"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# # convert categorical variable to numerical by replacing - könnte wichtig für die Erstellung von Histogrammen sein! Beispiel
rhc_all_nowell_df.DATENBANKART.replace(['asthma', 'covid', 'metabolic'], [0, 1, 2], inplace=True)
# Verteilung Ursprungsdatenbank asthma = 0, covid = 1, metabolic = 2
rhc_all_nowell_df["DATENBANKART"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Verteilung tot/lebend (1/0)
rhc_all_nowell_df["LEBENDIG"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Verteilung Encounter-Classen ambulatory/inpatient/emergency/outpatient als 0/1/2/3
rhc_all_nowell_df.ENCOUNTERCLASS_ENC.replace(['ambulatory', 'inpatient', 'emergency', 'outpatient'], [0, 1, 2, 3], inplace=True)
rhc_all_nowell_df["ENCOUNTERCLASS_ENC"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

# Multivariate Analysis: Analyzing Relationships Between Variables

In [ ]:
# Generating the correlating matrix
corr = rhc_all_nowell_df.corr()
plt.figure(figsize=(7, 5))
# Generating the correlation heat-map
sns.heatmap(corr, annot=True)

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
csv.__version__

In [ ]:
sns.__version__

In [ ]:
# Überprüfen aller Variablen
# %pip freeze

# Calculate the Checksum of the Data Set

In [ ]:
rhc_diagn_df_hashes = hash_pandas_object(rhc_diagn_df)
diag_anz_df_hashes = hash_pandas_object(diag_anz_df)
rhc_diagn_df_hashes = hash_pandas_object(rhc_diagn_df)
diagn_enc_df_hashes = hash_pandas_object(diagn_enc_df)
pat_anz_nowellrechnung_df_hashes = hash_pandas_object(pat_anz_nowellrechnung_df)
diagn_encnowell_df_hashes = hash_pandas_object(diagn_encnowell_df)
rhc_enc_anz_df_hashes = hash_pandas_object(rhc_enc_anz_df)
rhc_enc_anz_df_hashes = hash_pandas_object(rhc_enc_anz_df)
rhc_all_df_hashes = hash_pandas_object(rhc_all_df)
rhc_all_nowell_df_hashes = hash_pandas_object(rhc_all_nowell_df)
groupbypat_df_hashes = hash_pandas_object(groupbypat_df)
rhc_rein01_df_hashes = hash_pandas_object(rhc_rein01_df)
rhc_rein02_df_hashes = hash_pandas_object(rhc_rein02_df)
rhc_rein03_df_hashes = hash_pandas_object(rhc_rein03_df)
rhc_rein04_df_hashes = hash_pandas_object(rhc_rein04_df)
rhc_rein05_df_hashes = hash_pandas_object(rhc_rein05_df)
rhc_rein06_df_hashes = hash_pandas_object(rhc_rein06_df)
rhc_rein_df_hashes = hash_pandas_object(rhc_rein_df)
anz_pat_dign_enc_df_hashes = hash_pandas_object(anz_pat_dign_enc_df)

In [ ]:
Liste_rhcedaall_hashes = [rhc_diagn_df_hashes, diag_anz_df_hashes, rhc_diagn_df_hashes, diagn_enc_df_hashes, pat_anz_nowellrechnung_df_hashes, diagn_encnowell_df_hashes, rhc_enc_anz_df_hashes, rhc_enc_anz_df_hashes, rhc_all_df_hashes, rhc_all_nowell_df_hashes, groupbypat_df_hashes, rhc_rein01_df_hashes, rhc_rein02_df_hashes, rhc_rein03_df_hashes, rhc_rein04_df_hashes, rhc_rein05_df_hashes, rhc_rein06_df_hashes, rhc_rein_df_hashes, anz_pat_dign_enc_df_hashes]

In [ ]:
print(Liste_rhcedaall_hashes)

## Value should be 
2650178518257614002

-3795146034381248713

2650178518257614002

7831896322564588659

-840543143014388252

1693621233367357631

-5022892886560696056

-5058732213369095038

-1621010955553152918

-7846124668401262891

-7151992359203372660

6239097838065681928

6239097838065681928

-2502359466387942033

1851482707534779390

2377980241770097181

9152482894650796171

-1250212140467252653

In [ ]:
print(rhc_diagn_df_hashes.sum())
print(diag_anz_df_hashes.sum())
print(rhc_diagn_df_hashes.sum())
print(diagn_enc_df_hashes.sum())
print(pat_anz_nowellrechnung_df_hashes.sum())
print(diagn_encnowell_df_hashes.sum())
print(rhc_enc_anz_df_hashes.sum())
print(rhc_all_df_hashes.sum())
print(rhc_all_nowell_df_hashes.sum())
print(groupbypat_df_hashes.sum())
print(rhc_rein01_df_hashes.sum())
print(rhc_rein02_df_hashes.sum())
print(rhc_rein03_df_hashes.sum())
print(rhc_rein04_df_hashes.sum())
print(rhc_rein05_df_hashes.sum())
print(rhc_rein06_df_hashes.sum())
print(rhc_rein_df_hashes.sum())
print(anz_pat_dign_enc_df_hashes.sum())